# Data Analysis - Generate Models that can project player Statistics
* Given past stats, and predicted values for the coming season, predict performance for the upcoming season
* explanatory variables are:
  * lvl 0: most simplistic model
  * lvl a: (deterministic) age, season year, yrs_in_league
  * lvl a: human chosen (by me) with simplistic model => this is the baseline (include historical pace)
  * lvl b: a + all historical variables
  * lvl c: b + (projected) g (max 82), mpg (max 48), historical eff ratio (with next year's roster but prev year's stats)
* Models:
  * lvl 1: linear regression (no polynomial bc it adds too many variables)
  * lvl 2: random forest (doesn't do as well as regression for some reason?)
  * lvl 3: gradient boosted trees  
* predicted values are: 
  * pts/g, 3pts, reb, ast, blk, stl, fg%, ft%, fta/g, to, g, mp_per_g

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import defaultdict

In [2]:
# To plot matplotlib figures inline on the notebook
%matplotlib inline

#from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.cross_validation import cross_val_score, train_test_split, KFold
from sklearn.grid_search import GridSearchCV

/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from luther_common import prep_data, prep_data_for_prediction, background_gradient, visualize_scores, check_overfit, visualize_factors_heat_map, categories

##### (cells to reuse when debugging)

In [4]:
#take a look @ columns available
# for col in X_df.columns:
#       print(col)

In [5]:
# make sure our data is not missing anything (see if there are any nans)
# X_train[X_train.isnull().any(axis=1)]

## Settings

In [6]:
# consistent seed for random state
seed = 9001

## Load and Sequester Data

In [7]:
# load our data:
#option: regular old dataset
# X_df = pd.read_csv('LEBRON_data.csv', index_col=0)
#option: feature engineered dataset
X_df = pd.read_csv('LEBRON_data_feng.csv', index_col=0)
y_df = pd.read_csv('LEBRON_target.csv', index_col=0)

In [8]:
#sequester 15% holdout data (since we will use CV on X and y)
# X_train, X_holdout, y_train, y_holdout \
# = train_test_split(X_df, y_df, test_size=0.15, random_state=seed)

In [9]:
#alternatively, holdout data from 2017
X_train = X_df[X_df['season_year_0_ya'] != 2017]
X_holdout = X_df[X_df['season_year_0_ya'] == 2017]
y_train = y_df[y_df['season_year'] != 2017]
y_holdout = y_df[y_df['season_year'] == 2017]

In [10]:
# #alternatively, if we want to produce final models
# X_train = X_df
# X_holdout = None
# y_train = y_df
# y_holdout = None

In [11]:
#check
# X_train.shape, X_holdout.shape, y_train.shape, y_holdout.shape

## Fit our models and Visualize performance

### Prep Data

In [ ]:
# prep data
category = 'pts_per_g'
level = 'D' #['A', B','C','D']
std_ready_X, ready_X, ready_y, std_train = prep_data(category,level,X_train,y_train)
std_hld_X, hld_X, hld_y, std_hold = prep_data(category,level,X_holdout,y_holdout)

In [ ]:
# set training data to split into 5 folds
kfold = KFold(len(ready_X), n_folds=5, shuffle=True, random_state=seed)

### Prep and Train final naive models

In [ ]:
# %%time
# print("Lasso Model:")
# params = {
#     "alpha": np.logspace(-4, 1, 20, base=10.0)#,
# #    "tol": [0.001],
# #    "max_iter": [2000]
# }

# #store our grid estimators
# estimators = {}
# standardizers = {}

# for category in categories:
#     level = 'B'
#     std_ready_X, ready_X, ready_y, standardizers[category] = prep_data(category,level,X_train,y_train)
#     kfold = KFold(len(ready_X), n_folds=5, shuffle=True, random_state=seed)
#     grid_est = GridSearchCV(Lasso(), param_grid=params, cv=kfold, verbose=5, n_jobs=-1)
#     grid_est.fit(std_ready_X, ready_y)
#     estimators[category] = grid_est

In [ ]:
# for key, grid_estimator in estimators.items():
#     print('Key: '+key)
#     print('Estimator:')
#     print(grid_estimator.best_estimator_)
#     result_df = pd.DataFrame(grid_estimator.grid_scores_)
#     print('Best Trained Score: {}'.format(np.max(result_df.mean_validation_score)))
#     print('Heat Map:')
#     visualize_factors_heat_map(grid_estimator,ready_X,'regression')

In [ ]:
# # pickle the models we've come up with
# from sklearn.externals import joblib
# for key, grid_estimator in estimators.items():
#     joblib.dump(grid_estimator.best_estimator_, 'naive_linreg_predictor_'+key+'.pkl')
#     # estimator = joblib.load('filename.pkl')
# for key, standardizer in standardizers.items():
#     joblib.dump(standardizer, 'naive_linreg_standardizer_'+key+'.pkl')
#     # estimator = joblib.load('filename.pkl')

### Prep and Train final 'best' models

In [13]:
# %%time
# print("Lasso Model:")
# params = {
#     "alpha": np.logspace(-4, 1, 40, base=10.0)#,
# #    "tol": [0.001],
# #    "max_iter": [2000]
# }

# #store our grid estimators
# estimators = {}
# standardizers = {}

# for category in categories:
#     level = '0'
#     std_ready_X, ready_X, ready_y, standardizers[category] = prep_data(category,level,X_train,y_train)
#     kfold = KFold(len(ready_X), n_folds=5, shuffle=True, random_state=seed)
#     grid_est = GridSearchCV(Lasso(), param_grid=params, cv=kfold, verbose=5, n_jobs=-1)
#     grid_est.fit(std_ready_X, ready_y)
#     estimators[category] = grid_est

Lasso Model:
Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] ........................... alpha=0.0001, score=0.751546 -   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................... alpha=0.0001, score=0.780365 -   0.1s
[CV] ........................... alpha=0.0001, score=0.778882 -   0.1s
[CV] alpha=0.00013433993326 ..........................................
[CV] alpha=0.00013433993326 ..........................................
[CV] ........................... alpha=0.0001, score=0.760531 -   0.0s
[CV] ........................... alpha=0.0001, score=0.728752 -   0.2s
[CV] ................. alpha=0.00013433993326, score=0.780364 -   0.0s
[C

[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.3s


[CV] alpha=0.0014251026703 ...........................................
[CV] alpha=0.00191448197617 ..........................................
[CV] ................ alpha=0.000587801607227, score=0.778880 -   0.0s
[CV] alpha=0.00257191380906 ..........................................
[CV] ................. alpha=0.00078965228685, score=0.751551 -   0.0s
[CV] alpha=0.00078965228685 ..........................................
[CV] ................. alpha=0.00191448197617, score=0.760525 -   0.0s
[CV] alpha=0.00257191380906 ..........................................
[CV] ................. alpha=0.00257191380906, score=0.780349 -   0.0s
[CV] alpha=0.00345510729459 ..........................................
[CV] .................. alpha=0.0014251026703, score=0.780356 -   0.0s
[CV] ................. alpha=0.00078965228685, score=0.760529 -   0.0s
[CV] alpha=0.0014251026703 ...........................................
[CV] alpha=0.00623550734127 ..........................................
[CV] .

[CV] .................. alpha=0.0888623816274, score=0.778236 -   0.0s
[CV] alpha=0.0888623816274 ...........................................
[CV] .................. alpha=0.0272833337649, score=0.751711 -   0.0s
[CV] alpha=0.0272833337649 ...........................................
[CV] ................... alpha=0.119377664171, score=0.779051 -   0.0s
[CV] alpha=0.119377664171 ............................................
[CV] .................. alpha=0.0492388263171, score=0.728912 -   0.0s
[CV] .................. alpha=0.0272833337649, score=0.760415 -   0.0s
[CV] alpha=0.0492388263171 ...........................................
[CV] alpha=0.0366524123708 ...........................................
[CV] ................... alpha=0.119377664171, score=0.777857 -   0.0s
[CV] .................. alpha=0.0366524123708, score=0.780085 -   0.0s
[CV] alpha=0.119377664171 ............................................
[CV] alpha=0.0366524123708 ...........................................
[CV] .

[CV] .................... alpha=1.70125427985, score=0.686913 -   0.0s
[CV] ................... alpha=0.942668455118, score=0.737801 -   0.0s
[CV] alpha=2.28546386413 .............................................
[CV] alpha=4.1246263829 ..............................................
[CV] .................... alpha=3.07029062976, score=0.532737 -   0.0s
[CV] .................... alpha=2.28546386413, score=0.631280 -   0.0s
[CV] .................... alpha=1.26638017347, score=0.714791 -   0.0s
[CV] alpha=3.07029062976 .............................................
[CV] alpha=1.26638017347 .............................................
[CV] alpha=2.28546386413 .............................................
[CV] .................... alpha=3.07029062976, score=0.513387 -   0.0s
[CV] ..................... alpha=4.1246263829, score=0.325219 -   0.0s
[CV] alpha=3.07029062976 .............................................
[CV] .................... alpha=1.26638017347, score=0.719721 -   0.0s
[CV] a

...

[CV] alpha=0.00623550734127 ..........................................
[CV] ................. alpha=0.00257191380906, score=0.387410 -   0.0s
[CV] alpha=0.00257191380906 ..........................................
[CV] ................... alpha=0.011253355826, score=0.408024 -   0.0s
[CV] ................. alpha=0.00623550734127, score=0.387205 -   0.0s
[CV] alpha=0.0151177507062 ...........................................
[CV] alpha=0.00623550734127 ..........................................
[CV] .................. alpha=0.0151177507062, score=0.404197 -   0.0s
[CV] ................. alpha=0.00106081835514, score=0.387342 -   0.0s
[CV] alpha=0.0151177507062 ...........................................
[CV] alpha=0.00106081835514 ..........................................
[CV] ................. alpha=0.00623550734127, score=0.335204 -   0.0s
[CV] ................. alpha=0.00257191380906, score=0.333973 -   0.0s
[CV] alpha=0.00257191380906 ..........................................
[CV] a

[CV] alpha=0.0661474064123 ...........................................
[CV] .................. alpha=0.0366524123708, score=0.384169 -   0.0s
[CV] .................. alpha=0.0661474064123, score=0.332036 -   0.0s
[CV] alpha=0.0661474064123 ...........................................
[CV] alpha=0.0366524123708 ...........................................
[CV] ................... alpha=0.160371874375, score=0.043815 -   0.0s
[CV] alpha=0.160371874375 ............................................
[CV] .................. alpha=0.0366524123708, score=0.370524 -   0.0s
[CV] alpha=0.0366524123708 ...........................................
[CV] .................. alpha=0.289426612472, score=-0.001213 -   0.0s
[CV] ................... alpha=0.160371874375, score=0.038703 -   0.0s
[CV] alpha=0.388815518031 ............................................
[CV] alpha=0.215443469003 ............................................
[CV] .................. alpha=0.0366524123708, score=0.329733 -   0.0s
[CV] a

[CV] alpha=2.28546386413 .............................................
[CV] ................... alpha=2.28546386413, score=-0.000680 -   0.0s
[CV] alpha=2.28546386413 .............................................
[CV] .................... alpha=4.1246263829, score=-0.002012 -   0.0s
[CV] .................. alpha=0.942668455118, score=-0.000715 -   0.0s
[CV] alpha=0.942668455118 ............................................
[CV] alpha=4.1246263829 ..............................................
[CV] ................... alpha=2.28546386413, score=-0.000715 -   0.0s
[CV] ............................ alpha=10.0, score=-0.000077 -   0.0s
[CV] alpha=2.28546386413 .............................................
[CV] alpha=10.0 ......................................................
[CV] ................... alpha=2.28546386413, score=-0.001213 -   0.0s
[CV] alpha=3.07029062976 .............................................
[CV] .................. alpha=0.942668455118, score=-0.001213 -   0.0s
[CV] .

[Parallel(n_jobs=-1)]: Done 152 out of 200 | elapsed:    0.9s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    1.1s finished


CPU times: user 7.81 s, sys: 3.08 s, total: 10.9 s
Wall time: 20 s


In [14]:
# for key, grid_estimator in estimators.items():
#     print('Key: '+key)
#     print('Estimator:')
#     print(grid_estimator.best_estimator_)
#     result_df = pd.DataFrame(grid_estimator.grid_scores_)
#     print('Best Trained Score: {}'.format(np.max(result_df.mean_validation_score)))
#     #print('Heat Map:')
#     #visualize_factors_heat_map(grid_estimator,ready_X,'regression')

Key: fg_per_g
Estimator:
Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
Best Trained Score: 0.7480889201278261
Key: pts_per_g
Estimator:
Lasso(alpha=0.00043754793750741842, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
Best Trained Score: 0.7600151263191718
Key: stl_per_g
Estimator:
Lasso(alpha=0.0003257020655659783, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
Best Trained Score: 0.6833937764417483
Key: ast_per_g
Estimator:
Lasso(alpha=0.0019144819761699576, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selectio

In [18]:
# # pickle the models we've come up with
# from sklearn.externals import joblib
# for key, grid_estimator in estimators.items():
#     joblib.dump(grid_estimator.best_estimator_, 'best_linreg_predictor_'+key+'.pkl')
#     # estimator = joblib.load('filename.pkl')
# for key, standardizer in standardizers.items():
#     joblib.dump(standardizer, 'best_linreg_standardizer_'+key+'.pkl')
#     # estimator = joblib.load('filename.pkl')

### Lasso model

In [ ]:
%%time
print("Lasso Model:")
params = {
    "alpha": np.logspace(-4, 1, 20, base=10.0)#,
#    "tol": [0.001],
#    "max_iter": [2000]
}

grid_est = GridSearchCV(Lasso(), param_grid=params, cv=kfold, verbose=5, n_jobs=-1)
grid_est.fit(std_ready_X, ready_y)

In [ ]:
check_overfit(grid_est, std_hld_X, hld_y)

In [ ]:
visualize_scores(grid_est,params,['alpha'])

In [ ]:
visualize_factors_heat_map(grid_est,ready_X,'regression')

### Polynomial Lasso model

In [ ]:
poly = PolynomialFeatures(degree=2)
ready_X_poly = poly.fit_transform(ready_X)
# standardize predictor variables
std = StandardScaler()
std.fit(ready_X_poly)
std_ready_X_poly = std.transform(ready_X_poly)

In [ ]:
%%time
print("Polynomial Lasso Model:")
params = {
    "alpha": np.logspace(-4, 1, 20, base=10.0)#,
#    "tol": [0.001],
#    "max_iter": [2000]
}

grid_est = GridSearchCV(Lasso(), param_grid=params, cv=kfold, verbose=5)
grid_est.fit(std_ready_X_poly, ready_y)

In [ ]:
hld_X_poly = poly.fit_transform(hld_X)
std_hld_X_poly = std.transform(hld_X_poly)
check_overfit(grid_est, std_hld_X_poly, hld_y)

In [ ]:
visualize_scores(grid_est,params,['alpha'])

In [ ]:
visualize_factors_heat_map(grid_est,ready_X,'poly_regression', poly=poly)

### Random Forest model

In [ ]:
%%time
print("Random Forest Model:")
params = {
    "n_estimators":[100,500,1000],
#    "n_estimators": np.logspace(3, 3.5, 2).astype(int),
#    "max_depth": np.linspace(4, 10, 4)
#    "max_features": np.linspace(0.25,0.35,3)
    "max_features": np.linspace(0.3,0.8,6)
}
grid_est = GridSearchCV(RandomForestRegressor(), param_grid=params, cv=kfold, verbose=5, n_jobs=-1)
grid_est.fit(ready_X, ready_y)

In [ ]:
check_overfit(grid_est, hld_X, hld_y)

In [ ]:
visualize_scores(grid_est,params,['n_estimators'])

In [ ]:
visualize_factors_heat_map(grid_est,ready_X,'tree')

### Build and test gradient boosted tree model

In [ ]:
%%time

print("Gradient Boosted Tree Model:")
params = {
    "n_estimators": [100,500,1000],
    "learning_rate": np.logspace(-2, -1, 3),
    "max_depth": [2,3,4]
}
grid_est = GridSearchCV(GradientBoostingRegressor(), param_grid=params, cv=kfold, verbose=5, n_jobs=-1)
grid_est.fit(ready_X, ready_y)

In [ ]:
check_overfit(grid_est, hld_X, hld_y)

In [ ]:
visualize_scores(grid_est,params,['n_estimators'])

In [ ]:
visualize_factors_heat_map(grid_est,ready_X,'tree')